In [ ]:
import random
import math
import matplotlib.pyplot as plt

In [3]:
class Displayable:
    max_dispaly_level = 1
    def display(self, level=0, *args, **nargs):
        if level <= self.max_dispaly_level:
            self.__display(level, *args, **nargs)

In [ ]:
class Plot_history(object):
    def __init__(self, ag, env):
        self.ag = ag
        self.env = env
        plt.ion()
        plt.xlabel('Time')
        plt.ylabel('Value')

    def plot_env_hist(self):
        num = len(self.env.history)


: 

In [ ]:
class PriceMonitoringController:
    """Monitors the price and determines if it is below the threshold."""
    def __init__(self, threshold_discount=0.2):
        self.threshold_discount = threshold_discount

    def is_price_below_threshold(self, current_price, average_price):
        """Check if the price is below the threshold."""
        return current_price < (1 - self.threshold_discount) * average_price


class InventoryMonitoringController:
    """Monitors inventory levels and checks if restocking is needed."""
    def __init__(self, critical_threshold=10):
        self.critical_threshold = critical_threshold

    def is_inventory_critical(self, current_stock):
        """Check if inventory is below the critical threshold."""
        return current_stock < self.critical_threshold


class OrderingController:
    """Decides the quantity to order based on inputs from other controllers."""
    def __init__(self):
        self.buy_history = []
        self.spent = 0

    def decide_order(self, current_price, average_price, current_stock, price_controller, inventory_controller):
        """Decide the quantity to order."""
        if price_controller.is_price_below_threshold(current_price, average_price) and current_stock >= 10:
            tobuy = 15
        elif inventory_controller.is_inventory_critical(current_stock):
            tobuy = 10
        else:
            tobuy = 0

        self.buy_history.append(tobuy)
        self.spent += tobuy * current_price
        return tobuy


class TP_agent(Agent):
    """Agent integrates all controllers and selects the action."""
    def __init__(self):
        percept = env.initial_percept()
        self.ave = self.last_price = percept['price']
        self.instock = percept['instock']

        # Initialize controllers
        self.price_controller = PriceMonitoringController(threshold_discount=0.2)
        self.inventory_controller = InventoryMonitoringController(critical_threshold=10)
        self.ordering_controller = OrderingController()

    def select_action(self, percept):
        """Return the next action to carry out."""
        self.last_price = percept['price']
        self.ave = self.ave + (self.last_price - self.ave) * 0.05
        self.instock = percept['instock']

        tobuy = self.ordering_controller.decide_order(
            self.last_price,
            self.ave,
            self.instock,
            self.price_controller,
            self.inventory_controller
        )
        return {'buy': tobuy}
